In [1]:
import json
import torch
import torch.nn as nn
import bitsandbytes as bnb
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model
from fuzzywuzzy import fuzz
import re

/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
cache_dir = "/cs/student/projects1/aibh/2024/tpatil/.cache/huggingface/"
model_name = "microsoft/Phi-3.5-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map='auto',
    trust_remote_code=True,
    cache_dir=cache_dir
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    cache_dir=cache_dir
    )

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]


In [3]:
for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [4]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable parameters: {trainable_params} || All parameters: {all_params} || Trainable parameters %: {trainable_params/all_params*100}")

In [5]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    # target_modules="all-linear",
    target_modules=['qkv_proj', 'o_proj', 'gate_up_proj', 'down_proj'],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

Trainable parameters: 25165824 || All parameters: 3846245376 || Trainable parameters %: 0.6542958532243159


In [6]:
# Assuming your model is already loaded
# for name, module in model.named_modules():
#     print(name)

In [7]:
def load_tatqa_dataset(path):
    with open(path, "r") as f:
        return json.load(f)

train_data = load_tatqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/TATQA/tatqa_dataset_train.json")
val_data = load_tatqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/TATQA/tatqa_dataset_dev.json")
test_data = load_tatqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/TATQA/tatqa_dataset_test_gold.json")

In [8]:
def create_prompt_instance(table, paragraphs, question, answer, derivation=None):
    set_context = "You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\n"
    table_prompt = "The table provide the financial data. All the elements in the table are separated by \"|\". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n"
    paragraph_prompt = "The paragraphs provides some context about the data in the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple paragraphs separated by keyword matching \"Paragraph [0-9]+:\". Interpret each paragraph and use the data and description in it to infer the answer to the provided quesions.\n"
    question_prompt = "The question is asked based on the data in the table and the paragraph. You are expected to answer the question based on the data in the table and the paragraph.\n"
    answer_prompt = "" #"You are expected to answer the question based on the data in the table and the paragraph. Provide only the answer to the question and do not repeat the question. Use the answers provided as labels to learn the correct way to answer the question.\n"
    # derivation_prompt = "The derivation provides the steps to calculate the answer to the question. You can learn how to use the derivation to calculate the answer to the question.\n"
    answer_instruction_prompt = "\nInstruction: Answer the question based on the data in the table and the paragraph. Provide only the answer to the question and do not repeat the question. Use the answers provided as labels to learn the correct way to answer the question.\n"

    table_prompt += f"\nTable:\n"
    for row in table["table"]:
        table_prompt += "|".join([str(cell) for cell in row]) + " \n"
    
    for paragraph in paragraphs:
        paragraph_prompt += f"\nParagraph {paragraph['order']}:\n"
        paragraph_prompt += paragraph['text'] + " "

    question_prompt += f"\nQuestion:\n {question}"
    answer_prompt += f"\nAnswer:\n {answer}"
    # derivation_prompt += f"\nDerivation:\n {derivation}"

    return set_context, table_prompt, paragraph_prompt, question_prompt, answer_prompt, answer_instruction_prompt #, derivation_prompt

In [9]:
class TATQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.qa_pairs = []

        for item in self.data:
            tables = item.get('table', [])
            paragraphs = item.get('paragraphs', [])
            questions = item.get('questions', [])

            for question_answer in questions:
                question = question_answer["question"].strip()
                answer = question_answer["answer"]
                # derivation = question_answer.get("derivation", "")

                set_context, table_prompt, paragraph_prompt, question_prompt, answer_prompt, answer_instruction_prompt = create_prompt_instance(tables, paragraphs, question, answer)
                input_context = (set_context + table_prompt + paragraph_prompt + question_prompt + answer_instruction_prompt).strip()
                label_text = answer_prompt.strip()

                self.qa_pairs.append((
                    input_context,
                    label_text
                ))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_context, label_text = self.qa_pairs[idx]

        inputs = self.tokenizer( 
                    input_context, 
                    # max_length=self.max_length, 
                    # truncation=True, 
                    # padding="max_length", 
                    return_tensors="pt"
                )

        labels = self.tokenizer(
                    label_text, 
                    # max_length=self.max_length, 
                    # truncation=True, 
                    # padding="max_length", 
                    return_tensors="pt"
                )

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": labels["input_ids"].squeeze(0)
        }

In [10]:
train_tatqa_dataset = TATQADataset(train_data, tokenizer)
val_tatqa_dataset = TATQADataset(val_data, tokenizer)
test_tatqa_dataset = TATQADataset(test_data, tokenizer)

train_dataloader = DataLoader(
    train_tatqa_dataset,
    batch_size=1, 
    shuffle=True
)

val_dataloader = DataLoader(
    val_tatqa_dataset,
    batch_size=1, 
    shuffle=False
)

test_dataloader = DataLoader(
    test_tatqa_dataset,
    batch_size=1,
    shuffle=False
)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="steps",
    # eval_steps=500,
    # save_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # gradient_accumulation_steps=1,
    warmup_steps=100,
    max_steps=200,
    weight_decay=0.01,
    learning_rate=2e-4,
    logging_dir="./logs",
    logging_steps=1,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    train_dataset=train_tatqa_dataset,
    eval_dataset=val_tatqa_dataset,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

/tmp/ipykernel_134229/908364913.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()
trainer.save_model("./results/tatqa_model")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
You are not running the flash-attention implementation, expect numerical differences.
/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.909600
2,1.824800
3,1.149800
4,1.914800
5,2.204000
6,1.763800
7,1.762000
8,1.871800
9,1.708400
10,2.058100


In [13]:
ft_model_path = "./results/tatqa_model"
lora_config = PeftConfig.from_pretrained(ft_model_path)
ft_model = PeftModel.from_pretrained(
    model, 
    ft_model_path, 
    config=lora_config,
    load_in_8bit=True,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float16
)
ft_model = ft_model.to(device)

/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.qkv_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.qkv_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.gate_up_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.gate_up_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.1.self_attn.o_proj.lora_A.default.weight', 'base

In [ ]:
target_count = 8
sample_count = 0

for batch in test_dataloader:
    if sample_count >= target_count:
            break
    
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    print("Input:", tokenizer.decode(input_ids.squeeze(0)))
    print("Label:", tokenizer.decode(labels.squeeze(0)))

    sample_count += 1

    

Input: You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.
The table provide the financial data. All the elements in the table are separated by "|". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.

Table:
Current assets|As Reported|Adjustments|Balances without Adoption of Topic 606 
Receivables, less allowance for doubtful accounts|$831.7|$8.7|$840.4 
Inventories .|1,571.7|(3.1)|1,568.6 
Prepaid expenses and other current assets|93.8|(16.6)|77.2 
Current liabili

In [ ]:
ft_model.eval()

generated_answers = []
reference_answers = []
reference_questions = []
target_count = 20
sample_count = 0

with torch.no_grad():
    for batch in test_dataloader:
        if sample_count >= target_count:
            break

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        batch_size = input_ids.shape[0]
        
        # Determine how many samples to take from this batch
        samples_needed = min(batch_size, target_count - sample_count)
        
        # Take only the needed samples from this batch
        batch_input_ids = input_ids[:samples_needed]
        batch_attention_mask = attention_mask[:samples_needed]
        batch_labels = labels[:samples_needed]
        
        # Decode questions
        batch_questions = tokenizer.batch_decode(batch_input_ids, skip_special_tokens=True)
        
        # Generate answers
        with torch.cuda.amp.autocast(enabled=True):
            outputs = ft_model.generate(
                input_ids=batch_input_ids,
                attention_mask=batch_attention_mask,
                max_new_tokens=64,
                # do_sample=False,
                num_beams=4,
                # top_k=10,
                # early_stopping=True,
                # pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
            )
        
        # Decode generated answers
        batch_answers = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for batch_answer in batch_answers:
            if ("Answer:" in batch_answer):
                batch_answer = re.search(r'(?<=Answer:\s).+?(?=\\n|\n|$)', batch_answer).group(0)
            generated_answers.append(batch_answer)
        
        # Add to our collections
        reference_questions.append(batch_questions)
        # generated_answers.extend(batch_answers)
        reference_answer = tokenizer.batch_decode(batch_labels, skip_special_tokens=True)
        reference_answer = re.search(r'(?<=Answer:\\n\s).+(?=(\"|\')\])', str(reference_answer)).group(0)
        list_element = re.search('(?<=\[).+(?=\])', str(reference_answer))
        if list_element:
            reference_answer = list_element.group(0).replace(", ", "; ").replace("'", "")
        reference_answers.append(reference_answer)
        
        # Update count
        sample_count += samples_needed

print(f"Collected {len(reference_questions)} samples:")
for i, (question, answer) in enumerate(zip(reference_questions, generated_answers)):
    print(f"\nSample {i+1}:")
    print(f"Q: {question}")
    print(f"A: {answer}")


/tmp/ipykernel_174216/2263786681.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Collected 15 samples:

Sample 1:
Q: ['You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe table provide the financial data. All the elements in the table are separated by "|". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n\nTable:\nCurrent assets|As Reported|Adjustments|Balances without Adoption of Topic 606 \nReceivables, less allowance for doubtful accounts|$831.7|$8.7|$840.4 \nInventories .|1,571.7|(3.1)|1,568.6 \nPrepaid expenses and other current as

In [199]:
reference_questions

[['You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe table provide the financial data. All the elements in the table are separated by "|". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n\nTable:\nCurrent assets|As Reported|Adjustments|Balances without Adoption of Topic 606 \nReceivables, less allowance for doubtful accounts|$831.7|$8.7|$840.4 \nInventories .|1,571.7|(3.1)|1,568.6 \nPrepaid expenses and other current assets|93.8|(16.6)|77.2 \nCurrent lia

In [20]:
generated_answers

['Modified retrospective method',
 '$0.5 million',
 'Inventories: 1,568.6, Other accrued liabilities: 690.5',
 '(16.6 / 93.8) * 100 = 17.72% ',
 'You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe table provide the financial data. All the elements in the table are separated by "|". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n\nTable:\nCurrent assets|As Reported|Adjustments|Balances without Adoption of Topic 606 \nReceivables, less allowance for doubtf

In [18]:
reference_answers

['the modified retrospective method',
 '$0.5 million',
 '1,568.6; 690.5',
 '17.7',
 '-0.2',
 '3.61',
 '2019; 2018; 2017',
 '6,577',
 '9,184',
 '273',
 '1',
 '2.8']

In [102]:
match = re.search(r'(?<=\[\').+(?=\'\])', str(reference_answers[0]))
print(match.group(0))

the modified retrospective method


In [68]:
batch_answers[0]

'You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe table provide the financial data. All the elements in the table are separated by "|". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n\nTable:\nCurrent assets|As Reported|Adjustments|Balances without Adoption of Topic 606 \nReceivables, less allowance for doubtful accounts|$831.7|$8.7|$840.4 \nInventories .|1,571.7|(3.1)|1,568.6 \nPrepaid expenses and other current assets|93.8|(16.6)|77.2 \nCurrent liabi

In [71]:
match = re.search(r'(?<=Answer:\s).+?(?=\n)', batch_answers[0])
print(match.group(0))


Modified retrospective method


In [21]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def clean_answer(text):
    """Extract and format numerical answer"""
#    if ':' in text:
#        text = text.split(':')[-1]

    # Handle percentages first
    # percent_match = re.search(r'[-+]?\d*\.?\d+\s*%?', text)
    percent_match = re.search(r'[-+]?\d*\.?\d+\s*%', text)
    if percent_match:
        number = float(percent_match.group(0).replace('%', '').strip())
        # If the number is small (likely decimal), convert to percentage
#        if abs(number) < 1:
#            number *= 100
        # Round to one decimal place and add % symbol
        # return f"{round(number, 0)}%"
        return str(number)

    # Handle regular numbers
    # decimal_match = re.search(r'[-+]?\d*,?\.?\d+', text, re.MULTILINE)
    decimal_match = re.search(r'(?<=\$)(\d{1,3}(,\d{3})*(\.\d+)?|\d+(\.\d+)?)', text)
    if decimal_match:
        match_str = decimal_match.group(0)
        if is_number(match_str):
            number = float(match_str)
            if abs(round(number) - number) < 0.01:
                return str(round(number))
        else:
            number = match_str
        # If it's close to an integer, round it
        # Otherwise, round to one decimal place
        return str(number)

    # Handle yes/no answers
    text = text.lower().strip()
    # if 'yes' in text or 'true' in text:
    #     return 'yes'
    # if 'no' in text or 'false' in text:
    #     return 'no'

    return text.strip()

In [166]:
for generated_answer in generated_answers:
    print(clean_answer(generated_answer))

modified retrospective method
0.5
inventories: 1,568.6, other accrued liabilities: 690.5
17.7
2.8
the ratio cannot be calculated as the total current liabilities balance, as reported, is not provided in the table or paragraph.
july 27, 2019; july 28, 2018; july 29, 2017
6,577
9,184
245


In [22]:
total_samples = 0
correct_predictions = 0
threshold = 85  # Minimum similarity percentage for correct match
similarity_scores = []
SETTING = "TATQA"

# import time

# start = time.time()

for i, generated_answer in enumerate(generated_answers):
    total_samples += 1
    clean_p = clean_answer(generated_answer)
    reference_answer = reference_answers[i]
    clean_e = clean_answer(reference_answer)


    # Compute similarity score
    # similarity = fuzz.ratio(predicted_answer.lower(), expected_answer.lower())
    similarity = fuzz.ratio(clean_p.lower(), clean_e.lower())
    similarity_scores.append(similarity)
    print()
    print()
    print("************************************************************************")
    print (f"*** Input {total_samples} *** ")
#    print(example["input_text"])
    print ("*** Expected answer *** ")
    print(reference_answer)
    print ("*** Clean Expected answer *** ")
    print(clean_e)
    print ("*** Predicted answer *** ")
    print(generated_answer)
    print ("*** Clean Predicted answer *** ")
    print(clean_p)

    # Count as correct if similarity is above threshold
    if similarity >= threshold:
        correct_predictions += 1

# Print Results
accuracy = correct_predictions / total_samples * 100
avg_similarity = sum(similarity_scores) / total_samples

print(f"*************************************{SETTING}***********************************")

print(f"Total Samples: {total_samples}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Average Similarity Score: {avg_similarity:.2f}%")

# Block of code to time
# end = time.time()

# print("Elapsed time:", end - start, "seconds")
# print("Avg Inf time:", (end - start)/dataset.shape[0], "seconds")



************************************************************************
*** Input 1 *** 
*** Expected answer *** 
the modified retrospective method
*** Clean Expected answer *** 
the modified retrospective method
*** Predicted answer *** 
Modified retrospective method
*** Clean Predicted answer *** 
modified retrospective method


************************************************************************
*** Input 2 *** 
*** Expected answer *** 
$0.5 million
*** Clean Expected answer *** 
0.5
*** Predicted answer *** 
$0.5 million
*** Clean Predicted answer *** 
0.5


************************************************************************
*** Input 3 *** 
*** Expected answer *** 
1,568.6; 690.5
*** Clean Expected answer *** 
1,568.6; 690.5
*** Predicted answer *** 
Inventories: 1,568.6, Other accrued liabilities: 690.5
*** Clean Predicted answer *** 
inventories: 1,568.6, other accrued liabilities: 690.5


************************************************************************
*** I

In [23]:
cache_dir = "/cs/student/projects1/aibh/2024/tpatil/.cache/huggingface/"
model_name = "microsoft/Phi-3.5-mini-instruct"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map='auto',
    trust_remote_code=True,
    cache_dir=cache_dir
    )

tokenizer_base_model = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    cache_dir=cache_dir
    )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]


In [26]:
base_model.eval()

generated_answers = []
reference_answers = []
reference_questions = []
target_count = 15
sample_count = 0

with torch.no_grad():
    for batch in test_dataloader:
        if sample_count >= target_count:
            break

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        batch_size = input_ids.shape[0]
        
        # Determine how many samples to take from this batch
        samples_needed = min(batch_size, target_count - sample_count)
        
        # Take only the needed samples from this batch
        batch_input_ids = input_ids[:samples_needed]
        batch_attention_mask = attention_mask[:samples_needed]
        batch_labels = labels[:samples_needed]
        
        # Decode questions
        batch_questions = tokenizer_base_model.batch_decode(batch_input_ids, skip_special_tokens=True)
        # Generate answers
        with torch.cuda.amp.autocast(enabled=True):
            outputs = base_model.generate(
                input_ids=batch_input_ids,
                attention_mask=batch_attention_mask,
                max_new_tokens=128,
                # do_sample=False,
                num_beams=4,
                # top_k=10,
                # early_stopping=True,
                # pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
            )
        
        # Decode generated answers
        batch_answers = tokenizer_base_model.batch_decode(outputs, skip_special_tokens=True)
        for batch_answer in batch_answers:
            if ("Answer:" in batch_answer):
                batch_answer = re.search(r'(?<=Answer:\s).+?(?=\\n|\n|$)', batch_answer).group(0)
            generated_answers.append(batch_answer)
        
        # Add to our collections
        reference_questions.append(batch_questions)
        # generated_answers.extend(batch_answers)
        reference_answer = tokenizer_base_model.batch_decode(batch_labels, skip_special_tokens=True)
        reference_answer = re.search(r'(?<=Answer:\\n\s).+(?=(\"|\')\])', str(reference_answer)).group(0)
        list_element = re.search('(?<=\[).+(?=\])', str(reference_answer))
        if list_element:
            reference_answer = list_element.group(0).replace(", ", "; ").replace("'", "")
        reference_answers.append(reference_answer)
        
        # Update count
        sample_count += samples_needed

print(f"Collected {len(reference_questions)} samples:")
for i, (question, answer) in enumerate(zip(reference_questions, generated_answers)):
    print(f"\nSample {i+1}:")
    print(f"Q: {question}")
    print(f"A: {answer}")


/tmp/ipykernel_174216/2111150306.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Collected 15 samples:

Sample 1:
Q: ['You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe table provide the financial data. All the elements in the table are separated by "|". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n\nTable:\nCurrent assets|As Reported|Adjustments|Balances without Adoption of Topic 606 \nReceivables, less allowance for doubtful accounts|$831.7|$8.7|$840.4 \nInventories .|1,571.7|(3.1)|1,568.6 \nPrepaid expenses and other current as

In [27]:
generated_answers

['Modified retrospective method',
 'You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe table provide the financial data. All the elements in the table are separated by "|". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n\nTable:\nCurrent assets|As Reported|Adjustments|Balances without Adoption of Topic 606 \nReceivables, less allowance for doubtful accounts|$831.7|$8.7|$840.4 \nInventories .|1,571.7|(3.1)|1,568.6 \nPrepaid expenses and other current asse

In [28]:
reference_answers

['the modified retrospective method',
 '$0.5 million',
 '1,568.6; 690.5',
 '17.7',
 '-0.2',
 '3.61',
 '2019; 2018; 2017',
 '6,577',
 '9,184',
 '273',
 '1',
 '2.8',
 'primary components of the deferred tax assets and liabilities',
 '948,578',
 '2019; 2018']